In [8]:
import requests
from bs4 import BeautifulSoup
import json
import os

# Base URL of the website
url = 'https://www.lnh.edu.pk/Doctors?spe=207825fb-4313-43ca-9398-398ed6fbb0cf' 

# Send a request to the page and get the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links to doctor detail pages
doctor_links = [a.get('href') for a in soup.find_all('a', class_='btn') if a.get('href')]

# List to store the doctor details
doctor_details = []

# Loop through each doctor link
for link in doctor_links:
    doctor_url = 'https://www.lnh.edu.pk' + link
    
    doctor_response = requests.get(doctor_url)
    doctor_soup = BeautifulSoup(doctor_response.content, 'html.parser')
    
    name_tag = doctor_soup.find('h3', class_='mt-0')
    name = name_tag.text.strip() if name_tag else 'N/A'

    qualifications_tag = doctor_soup.find('h5', class_='text-theme-colored')
    qualifications = qualifications_tag.text.strip() if qualifications_tag else 'N/A'

    # Locate the department information after the <br> tag
    # department_tag = doctor_soup.find('h5', class_='text-theme-colored')
    # if department_tag:
    #     br_tag = department_tag.find('br')
    #     department = br_tag.next_sibling.strip() 
    # else:
    #     department = 'N/A'
    department= 'Neurology'
   
    
    # Extract clinic details (e.g., location, contact, timings, etc.)
    clinics = []
    
    panels = doctor_soup.find_all('div', class_='panel')
    for panel in panels:
        clinic_name_tag = panel.find('div', class_='panel-title')
        clinic_name = clinic_name_tag.text.strip() if clinic_name_tag else 'N/A'
        
        location_tag = panel.find('span', class_='font-weight-700')
        location = location_tag.find_next_sibling('span').text.strip() if location_tag else 'N/A'
        
        contact_tag = panel.find('span', text='Contact:')
        contact = contact_tag.find_next_sibling('span').text.strip() if contact_tag else 'N/A'
        
        timings = []
        rows = panel.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= 3:
                day = cols[0].text.strip()
                shift = cols[1].text.strip()
                timing = cols[2].text.strip()
                timings.append({'day': day, 'shift': shift, 'timing': timing})
        
        clinics.append({
            'clinic_name': clinic_name,
            'location': location,
            'contact': contact,
            'timings': timings
        })
    
    doctor_details.append({
        'name': name,
        'qualifications': qualifications,
        'department': department,
        'hospital': 'Liaquat National Hospital',
        'clinics': clinics
    })

# Save the data to a JSON file
with open('doctor_details.json', 'w') as file:
    json.dump(doctor_details, file, indent=4)

# Print the extracted data
for doctor in doctor_details:
    print(doctor)


/tmp/ipykernel_7223/945340429.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contact_tag = panel.find('span', text='Contact:')


{'name': 'Dr. Feroza Saleem', 'qualifications': 'MBBS, FCPS', 'department': 'Neurology', 'hospital': 'Liaquat National Hospital', 'clinics': [{'clinic_name': 'Neurology Specialized Clinic', 'location': 'Nazimabad', 'contact': '021-36615855', 'timings': [{'day': 'Thu', 'shift': 'Evening', 'timing': '05:00 pm  to  06:30 pm'}]}]}
{'name': 'Dr. Hazim M. Yousuf Brohi', 'qualifications': 'FCPS (Neurology)', 'department': 'Neurology', 'hospital': 'Liaquat National Hospital', 'clinics': [{'clinic_name': 'Neurology Clinic', 'location': 'Liaquat National Hospital', 'contact': '02134412531, 02134412534', 'timings': [{'day': 'Mon', 'shift': 'Morning', 'timing': '09:00 am  to  01:00 pm'}, {'day': 'Wed', 'shift': 'Afternoon', 'timing': '01:30 pm  to  02:59 pm'}, {'day': 'Thu', 'shift': 'Morning', 'timing': '09:00 am  to  01:00 pm'}, {'day': 'Thu', 'shift': 'Evening', 'timing': '04:00 pm  to  06:00 pm'}]}, {'clinic_name': 'Neurology Specialized Clinic', 'location': 'Liaquat National Hospital', 'conta